In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:

!kaggle datasets download princelv84/dogsvscats

Dataset URL: https://www.kaggle.com/datasets/princelv84/dogsvscats
License(s): MIT
 83% 451M/545M [00:02<00:00, 105MB/s]
100% 545M/545M [00:02<00:00, 199MB/s]


In [ ]:
!unzip /content/dogsvscats.zip -d /content/

In [ ]:
#Import all the needed liabraries
import matplotlib.pyplot as plt
from PIL import Image
import random
import os

import tensorflow as tf

from tensorflow.keras import layers,models
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.utils import plot_model

from sklearn.metrics import confusion_matrix,classification_report,roc_curve,auc
import seaborn as sns

In [ ]:
cat_dir_path='/content/train/cats'
dog_dir_path='/content/train/dogs'

In [ ]:
len(os.listdir(cat_dir_path))

10000

In [ ]:

len(os.listdir(dog_dir_path))

10000

In [ ]:
base_dir_train='/content/train'
base_dir_test='/content/test'

img_width,img_height=227,227 # AlexNet input size
batch_size=32

validation_split=0.2


In [ ]:
#Load Dataset
train_dataset=tf.keras.utils.image_dataset_from_directory(
    base_dir_train,
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=validation_split,
    seed =123,
    subset='training'
)

Found 20000 files belonging to 2 classes.
Using 16000 files for training.


In [ ]:
validation_dataset=tf.keras.utils.image_dataset_from_directory(
    base_dir_train,
    label_mode='binary',
    image_size=(img_width,img_height),
    batch_size=batch_size,
    subset='validation',
    seed=123,
    validation_split=validation_split
)

Found 20000 files belonging to 2 classes.
Using 4000 files for validation.


In [ ]:
#Data Augmentation
data_augmentation=tf.keras.Sequential([

    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2)
])

In [ ]:
#Apply Data Augmentation On trianing data
t_dataset=train_dataset.map(lambda x,y:(data_augmentation(x,training=True),y))


In [ ]:


#Prefetch the data for performance
AUTOTUNE=tf.data.AUTOTUNE # thsi will check how much ram and cpu cores i have and will decide the optimal buffersize
t_dataset=t_dataset.prefetch(buffer_size=AUTOTUNE)# this will create the buffer for training
v_dataset=validation_dataset.prefetch(buffer_size=AUTOTUNE)

"""
with this prefetch cpu will load the next batch of image and prepare it while gpu is working on the current batch so both cpu and gpu is working
and no is sitting idle
"""


'\nwith this prefetch cpu will load the next batch of image and prepare it while gpu is working on the current batch so both cpu and gpu is working \nand no is sitting idle\n'

**Building AlexNet Model**

In [ ]:
alexnet_model=models.Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(96,(11,11),strides=4,activation='relu'),
    layers.MaxPooling2D(pool_size=(3,3),strides=2),

    layers.Conv2D(256,(5,5),padding='same',activation='relu'),
    layers.MaxPooling2D(pool_size=(3,3),strides=2),

    layers.Conv2D(384,(3,3),padding='same',activation='relu'),
    layers.Conv2D(384,(3,3),padding='same',activation='relu'),
    layers.Conv2D(256,(3,3),padding='same',activation='relu'),

    layers.MaxPooling2D(pool_size=(3,3),strides=2),

    layers.Flatten(),

    layers.Dense(4096,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4096,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1,activation='sigmoid')

])

alexnet_model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_5 (Rescaling)         │ (None, 227, 227, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 55, 55, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,285,441 (222.34 MB)

 Trainable params: 58,285,441 (222.34 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#In original alexnet there was LRN(Local Response Normalization) but we are not using it

In [ ]:
alexnet_model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])

In [ ]:
history=alexnet_model.fit(t_dataset,validation_data=v_dataset,epochs=5)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 211s 409ms/step - accuracy: 0.5074 - loss: 0.6942 - val_accuracy: 0.5415 - val_loss: 0.6792
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 207s 414ms/step - accuracy: 0.5637 - loss: 0.6792 - val_accuracy: 0.6633 - val_loss: 0.6257
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 214s 427ms/step - accuracy: 0.6450 - loss: 0.6333 - val_accuracy: 0.6913 - val_loss: 0.5850
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 210s 419ms/step - accuracy: 0.6829 - loss: 0.5885 - val_accuracy: 0.7390 - val_loss: 0.5250
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 259s 414ms/step - accuracy: 0.7275 - loss: 0.5378 - val_accuracy: 0.7362 - val_loss: 0.5238


In [ ]:
test_dataset=tf.keras.utils.image_dataset_from_directory(
    base_dir_test,
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,

    seed =123

)

Found 5000 files belonging to 2 classes.


In [ ]:
#Evaluate the model
test_loss,test_acc=alexnet_model.evaluate(test_dataset)
print(f"test_accuracy: {test_acc*100:.2f}%")

157/157 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - accuracy: 0.7346 - loss: 0.5124
test_accuracy: 73.70%


**How to save the model and load it back**

In [ ]:
alexnet_model.save('save_model_5_epochs.keras')

In [ ]:
"""
We can use model.save_weights too but that will save weights only and not the architecture and optimizer.
So if we want to train the saved weights again in some other notebook we will have to write the code again for the
architecture. cause we won't be able to tell if we were using maxpooling or avgpooling and likewise.
But if we have saved our model we can use that model in some other device too.
"""

"\nWe can use model.save_weights too but that will save weights only and not the architecture and optimizer.\nSo if we want to train the saved weights again in some other notebook we will have to write the code again for the \narchitecture. cause we won't be able to tell if we were using maxpooling or avgpooling and likewise. \nBut if we have saved our model we can use that model in some other device too.\n"

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
loaded_model=load_model('save_model_5_epochs.keras')

In [ ]:
#Now we can use this loaded model for inferring or further training

In [ ]:
history1=loaded_model.fit(t_dataset,validation_data=v_dataset,epochs=2)

Epoch 1/2
500/500 ━━━━━━━━━━━━━━━━━━━━ 213s 414ms/step - accuracy: 0.7567 - loss: 0.5017 - val_accuracy: 0.8020 - val_loss: 0.4389
Epoch 2/2
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 402ms/step - accuracy: 0.7775 - loss: 0.4660 - val_accuracy: 0.8185 - val_loss: 0.4033


In [ ]:
history1_dict=history1.history
history1_dict

{'accuracy': [0.7598749995231628, 0.7837499976158142],
 'loss': [0.49374476075172424, 0.4569380581378937],
 'val_accuracy': [0.8019999861717224, 0.8184999823570251],
 'val_loss': [0.43887951970100403, 0.4033077657222748]}